# Processing of Files for the MANCHI Station

Test notebook for data processing

## Libraries

In [1]:
%load_ext autoreload
%autoreload 2
import gnssvod as gv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pdb
import importlib
import zipfile
import os
import xarray as xr
import glob
import shutil
from gnssvod.hemistats.hemistats import hemibuild
#import georinex as gr
#import qgrid as interactive table 
from matplotlib.collections import PatchCollection
import matplotlib.dates as mdates
import shutil
from matplotlib import rcParams
from datetime import datetime 
import shutil
from pathlib import Path


## Processing 25O


This chunk searches for all files with the .25O extension and copies them into a folder to be processed as nc

In [18]:
# Root folder where the decompressed sessions are located
ROOT = Path(r"C:/Users/varga/Box/Data/RinexZipFiles/ACOPAN/2025-05-25")
# Output folder where all the .25O files will be copied
DEST = ROOT.parent / "ACOPAN_nc"

# Create destination folder if it does not exist
DEST.mkdir(exist_ok=True)

for session_dir in ROOT.iterdir():
    if not session_dir.is_dir():
        continue
    raw_dirs = [d for d in session_dir.iterdir() if d.is_dir() and "RINEX_3_03" in d.name]
    for raw_dir in raw_dirs:
        obs_files = list(raw_dir.glob("*.25O"))
        for f in obs_files:
            dest_file = DEST / f.name
            print(f"Copying {f} -> {dest_file}")
            shutil.copy2(f, dest_file)

print("Done. All .25O files were copied to:", DEST)


Copying C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\2025-05-25\ACOPAN_20250413150519\ACOPAN_raw_20250413150519_RINEX_3_03\ACOPAN_raw_20250413150519.25O -> C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\ACOPAN_nc\ACOPAN_raw_20250413150519.25O
Copying C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\2025-05-25\ACOPAN_20250414150520\ACOPAN_raw_20250414150520_RINEX_3_03\ACOPAN_raw_20250414150520.25O -> C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\ACOPAN_nc\ACOPAN_raw_20250414150520.25O
Copying C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\2025-05-25\ACOPAN_20250415150521\ACOPAN_raw_20250415150521_RINEX_3_03\ACOPAN_raw_20250415150521.25O -> C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\ACOPAN_nc\ACOPAN_raw_20250415150521.25O
Copying C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\2025-05-25\ACOPAN_20250415212838\ACOPAN_raw_20250415212838_RINEX_3_03\ACOPAN_raw_20250415212838.25O -> C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\ACOPAN_nc\ACOPAN_raw_20250415212838.25O
Copying C:\Users\varga\Box\Data\Rine

## Presets

### Time intervals

In [2]:
# Define the start day
startday = pd.to_datetime('25-04-2025', format='%d-%m-%Y')
# Generate a range of datetime values
timeintervals=pd.interval_range(start=startday, periods=10, freq='D', closed='left')
timeintervals

IntervalIndex([[2025-04-25 00:00:00, 2025-04-26 00:00:00),
               [2025-04-26 00:00:00, 2025-04-27 00:00:00),
               [2025-04-27 00:00:00, 2025-04-28 00:00:00),
               [2025-04-28 00:00:00, 2025-04-29 00:00:00),
               [2025-04-29 00:00:00, 2025-04-30 00:00:00),
               [2025-04-30 00:00:00, 2025-05-01 00:00:00),
               [2025-05-01 00:00:00, 2025-05-02 00:00:00),
               [2025-05-02 00:00:00, 2025-05-03 00:00:00),
               [2025-05-03 00:00:00, 2025-05-04 00:00:00),
               [2025-05-04 00:00:00, 2025-05-05 00:00:00)],
              dtype='interval[datetime64[ns], left]')

## Process Rinex Files

In [16]:
root = Path(r"C:\Users\varga\Box\Data\nc Files\ACOPAN_nc")
files = list(root.glob("*.*O"))

print("Found", len(files), "files:")
for f in files:
    print(f.name)


Found 44 files:
ACOPAN_raw_20250413150519.25O
ACOPAN_raw_20250414150520.25O
ACOPAN_raw_20250415150521.25O
ACOPAN_raw_20250415212838.25O
ACOPAN_raw_20250416212840.25O
ACOPAN_raw_20250417212841.25O
ACOPAN_raw_20250418212841.25O
ACOPAN_raw_20250419212843.25O
ACOPAN_raw_20250420212844.25O
ACOPAN_raw_20250421212844.25O
ACOPAN_raw_20250422212845.25O
ACOPAN_raw_20250423212846.25O
ACOPAN_raw_20250424212847.25O
ACOPAN_raw_20250425212848.25O
ACOPAN_raw_20250426212849.25O
ACOPAN_raw_20250427212850.25O
ACOPAN_raw_20250428212851.25O
ACOPAN_raw_20250429212852.25O
ACOPAN_raw_20250430212853.25O
ACOPAN_raw_20250501212854.25O
ACOPAN_raw_20250502212855.25O
ACOPAN_raw_20250503212856.25O
ACOPAN_raw_20250504212857.25O
ACOPAN_raw_20250505212858.25O
ACOPAN_raw_20250506212859.25O
ACOPAN_raw_20250507151751.25O
ACOPAN_raw_20250508151752.25O
ACOPAN_raw_20250509151752.25O
ACOPAN_raw_20250510151753.25O
ACOPAN_raw_20250511151753.25O
ACOPAN_raw_20250512151754.25O
ACOPAN_raw_20250513151755.25O
ACOPAN_raw_2025051415175

In [22]:
# 1) Input files (RINEX .25O) for ACOPAN
pattern = {
    'ACOPAN': str(DEST / '*.25O')
    # If you had mixed extensions, use: str(DEST / '*.*O')
    # If files can be in subfolders, use: str(DEST / '**' / '*.25O')
}

# 2) Output folder where NetCDF files will be written
outdir = Path(r'C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\2025-05-25\processed')
outdir.mkdir(parents=True, exist_ok=True)  # create the folder if it does not exist
outputdir = {'ACOPAN': str(outdir)}

# (Optional) quick sanity check
files = glob.glob(pattern['ACOPAN'])
print(f'Found {len(files)} .25O files (showing up to 5):', files[:5])
if len(files) == 0:
    raise FileNotFoundError("No .25O files found in DEST folder. Check DEST and file existence.")

# 3) Variables to keep (SNR)
keepvars = ['S?', 'S??']

# 4) Preprocessing
result = gv.preprocess(
    pattern,
    interval='15s',
    keepvars=keepvars,
    outputdir=outputdir,
    outputresult=True  # also return result in memory
)

# 5) Safe extraction of the first processed dataset from ACOPAN (xarray.Dataset)
acopan_res = result.get('ACOPAN', [])
if isinstance(acopan_res, list):
    if not acopan_res:
        raise ValueError("No datasets produced for 'ACOPAN'. "
                         "Check the pattern and/or keepvars.")
    obs = acopan_res[0]
elif isinstance(acopan_res, xr.Dataset):
    obs = acopan_res
else:
    raise TypeError(f"Unexpected return type for 'ACOPAN': {type(acopan_res)}")

obs


Found 44 .25O files (showing up to 5): ['C:\\Users\\varga\\Box\\Data\\RinexZipFiles\\ACOPAN\\ACOPAN_nc\\ACOPAN_raw_20250413150519.25O', 'C:\\Users\\varga\\Box\\Data\\RinexZipFiles\\ACOPAN\\ACOPAN_nc\\ACOPAN_raw_20250414150520.25O', 'C:\\Users\\varga\\Box\\Data\\RinexZipFiles\\ACOPAN\\ACOPAN_nc\\ACOPAN_raw_20250415150521.25O', 'C:\\Users\\varga\\Box\\Data\\RinexZipFiles\\ACOPAN\\ACOPAN_nc\\ACOPAN_raw_20250415212838.25O', 'C:\\Users\\varga\\Box\\Data\\RinexZipFiles\\ACOPAN\\ACOPAN_nc\\ACOPAN_raw_20250416212840.25O']
Created a temporary directory at C:\Users\varga\AppData\Local\Temp\tmpftmbceon
Could not find any files matching the pattern C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\2025-05-25\processed*.nc
C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\ACOPAN_nc\ACOPAN_raw_20250413150519.25O exists | Reading...
Processed 0 out of 3285505 lines (0.0%)
Processed 252956 out of 3285505 lines (7.7%)
Processed 557826 out of 3285505 lines (17.0%)
Processed 772245 out of 3285505 lines (23.5%)
Pro

GFZ0MGXRAP_20251030000_01D_05M_ORB.SP3.gz: 1.03MB [00:03, 309kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2362//GFZ0MGXRAP_20251030000_01D_05M_ORB.SP3.gz
C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251030000_01D_05M_ORB.SP3 file is read in 9.48 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251040000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2362//GFZ0MGXRAP_20251040000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20251040000_01D_05M_ORB.SP3.gz: 1.03MB [00:03, 328kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2362//GFZ0MGXRAP_20251040000_01D_05M_ORB.SP3.gz
C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251040000_01D_05M_ORB.SP3 file is read in 9.77 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251030000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20251030000_01D_30S_CLK.CLK.gz

...

 | Download completed for GFZ0MGXRAP_20251440000_01D_30S_CLK.CLK.gz
C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251440000_01D_30S_CLK.CLK file is read in 11.64 seconds
SP3 interpolation is done in 5.49 seconds
Saved 105864 individual observations in C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\2025-05-25\processed\ACOPAN_raw_20250523151803.nc
C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\ACOPAN_nc\ACOPAN_raw_20250524151803.25O exists | Reading...
Observation file  C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\ACOPAN_nc\ACOPAN_raw_20250524151803.25O  is read in 3.85 seconds.
Processing 103112 individual observations
Calculating Azimuth and Elevation
C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251440000_01D_05M_ORB.SP3 exists | Reading...
C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251440000_01D_05M_ORB.SP3 file is read in 4.81 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251450000_01D_05M_ORB.SP3


GFZ0MGXRAP_20251450000_01D_05M_ORB.SP3.gz: 1.05MB [00:03, 331kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2368//GFZ0MGXRAP_20251450000_01D_05M_ORB.SP3.gz
C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251450000_01D_05M_ORB.SP3 file is read in 8.87 seconds
C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251440000_01D_30S_CLK.CLK exists | Reading...
C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251440000_01D_30S_CLK.CLK file is read in 15.49 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251450000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20251450000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20251450000_01D_30S_CLK.CLK.gz: 4.80MB [00:03, 1.31MB/s]                            


 | Download completed for GFZ0MGXRAP_20251450000_01D_30S_CLK.CLK.gz
C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251450000_01D_30S_CLK.CLK file is read in 2.27 seconds
SP3 interpolation is done in 3.77 seconds
Saved 103112 individual observations in C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\2025-05-25\processed\ACOPAN_raw_20250524151803.nc
C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\ACOPAN_nc\ACOPAN_raw_20250525151804.25O exists | Reading...
Observation file  C:\Users\varga\Box\Data\RinexZipFiles\ACOPAN\ACOPAN_nc\ACOPAN_raw_20250525151804.25O  is read in 0.72 seconds.
Processing 12687 individual observations
Calculating Azimuth and Elevation
C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251450000_01D_05M_ORB.SP3 exists | Reading...
C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251450000_01D_05M_ORB.SP3 file is read in 0.83 seconds
C:\Users\varga\AppData\Local\Temp\tmpftmbceon\GFZ0MGXRAP_20251450000_01D_30S_CLK.CLK exists | Reading...
C:\Users

In [ ]:
obs = result['ACOPAN'][0]
obs.epoch

KeyError: 'MANCHI'

In [20]:
obs.observation

S1C  S1X   S2C   S2I  S2X  S7I  S7X     Azimuth  \
Epoch               SV                                                      
2025-04-20 22:33:30 C23   NaN  NaN   NaN  20.0  NaN  NaN  NaN  166.650442   
                    C27   NaN  NaN   NaN  48.0  NaN  NaN  NaN  -18.417633   
                    C28   NaN  NaN   NaN  40.0  NaN  NaN  NaN   51.844026   
                    C32   NaN  NaN   NaN  36.0  NaN  NaN  NaN -119.403873   
                    C33   NaN  NaN   NaN  17.0  NaN  NaN  NaN   30.431649   
...                       ...  ...   ...   ...  ...  ...  ...         ...   
2025-04-21 22:33:00 R08  27.0  NaN  26.0   NaN  NaN  NaN  NaN   16.126631   
                    R12  19.0  NaN   NaN   NaN  NaN  NaN  NaN -167.267077   
                    R22  42.0  NaN  33.0   NaN  NaN  NaN  NaN   92.813625   
                    R23  33.0  NaN   NaN   NaN  NaN  NaN  NaN         NaN   
                    R24  37.0  NaN  29.0   NaN  NaN  NaN  NaN  -23.953310   

                         Elevation  
Epoch               SV              
2025-04-20 22:33:30 C23  10.878548  
                    C27  39.076360  
                    C28  63.342841  
                    C32  45.994334  
                    C33  18.276292  
...                            ...  
2025-04-21 22:33:00 R08  39.955684  
                    R12   3.436013  
                    R22  34.655140  
                    R23        NaN  
                    R24   8.772009  

[95433 rows x 9 columns]

## Save NCs

In [21]:

pattern = {
    'MANCHI': r'C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\Last\MANCHI_20250420223245\MANCHI_raw_20250420223245_RINEX_3_03\MANCHI_raw_20250420223245.25O'
}
outputdir = {
    'MANCHI': r'C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\nc\\'
}
result = gv.preprocess(
    pattern,
    interval='15s',
    keepvars=['S?','S??'],   # SNR típicamente
    outputdir=outputdir,
    outputresult=True,
    # approx_position=approx_position
)

Created a temporary directory at C:\Users\varga\AppData\Local\Temp\tmparjp82yl
Could not find any files matching the pattern C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\nc\\*.nc
C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\Last\MANCHI_20250420223245\MANCHI_raw_20250420223245_RINEX_3_03\MANCHI_raw_20250420223245.25O exists | Reading...
Observation file  C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\Last\MANCHI_20250420223245\MANCHI_raw_20250420223245_RINEX_3_03\MANCHI_raw_20250420223245.25O  is read in 3.66 seconds.
Processing 95 433 individual observations
Calculating Azimuth and Elevation
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3.gz: 1.05MB [00:03, 340kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3.gz
C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251100000_01D_05M_ORB.SP3 file is read in 7.27 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251110000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251110000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20251110000_01D_05M_ORB.SP3.gz: 1.04MB [00:03, 330kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251110000_01D_05M_ORB.SP3.gz
C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251110000_01D_05M_ORB.SP3 file is read in 13.72 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251120000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2363//GFZ0MGXRAP_20251120000_01D_05M_ORB.SP3.gz

...

 | Download completed for GFZ0MGXRAP_20251100000_01D_30S_CLK.CLK.gz
C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251100000_01D_30S_CLK.CLK file is read in 4.02 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251110000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20251110000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20251110000_01D_30S_CLK.CLK.gz: 4.80MB [00:03, 1.30MB/s]                            


 | Download completed for GFZ0MGXRAP_20251110000_01D_30S_CLK.CLK.gz
C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251110000_01D_30S_CLK.CLK file is read in 31.18 seconds
This file does not exist: C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251120000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20251120000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20251120000_01D_30S_CLK.CLK.gz: 4.80MB [00:04, 1.04MB/s]                            


 | Download completed for GFZ0MGXRAP_20251120000_01D_30S_CLK.CLK.gz
C:\Users\varga\AppData\Local\Temp\tmparjp82yl\GFZ0MGXRAP_20251120000_01D_30S_CLK.CLK file is read in 4.97 seconds
SP3 interpolation is done in 33.78 seconds
Saved 95 433 individual observations in C:\Users\varga\Box\Data\RinexZipFiles\MANCHI\nc\MANCHI_raw_20250420223245.nc
Removed the temporary directory at C:\Users\varga\AppData\Local\Temp\tmparjp82yl
